## Necessary packages are pandas and numpy

In [ ]:
#Cell 1

import pandas as pd
import numpy as np

## Roadway EEC

In [ ]:
#Cell 2
#Importing the CSV where the projects have been spatially joined with the roadway EEC as a dataframe


Roadway_EEC = pd.read_csv ("Roadway_EEC.csv", encoding ="latin-1")
Roadway_EEC.head ()

In [ ]:
#Cell 3
#Adding a new column, Length. Segment Leghth = END_MP - BEGIN_MP

Roadway_EEC ["LENGTH"]=Roadway_EEC ["END_MP"]-Roadway_EEC["BEGIN_MP"]
Roadway_EEC.head (2)

In [ ]:
#Cell 4

#If the beginning milepoint is in the middle of a segment, it will get "S"
#If the ending milepoint is in the middle of a segment, it will get "E"
#If the beginning and ending mileipoint both fall in between a segment, it will get "M"
#Otherwise it will get "0"


def start_end(x):
    if x.BEGINNINGMILE>x.BEGIN_MP and x.ENDINGMILE <x.END_MP:
        return "M"
    elif x.BEGINNINGMILE>=x.BEGIN_MP and x.BEGINNINGMILE<=x.END_MP:
        return "S"
    elif x.ENDINGMILE>=x.BEGIN_MP and x.ENDINGMILE<=x.END_MP:
        return "E"
    else:
        return "O"

In [ ]:
#Cell 5
#Creating new column for "S", "E", "M" and "O"

Roadway_EEC['StartEnd']=Roadway_EEC.apply (start_end, axis = 1)
Roadway_EEC.head ()

In [ ]:
#Cell 6

#Calculating the new length for which the EEC is needed.
#"S">> new_length = END_MP of segment - Beginning milepoint of project
#"E">> new_length = Ending milepoint of project - BEGIN_MP of the segment
# "M" >> new_length = Ending milepoint of project - Beginning milepoint of project
# "O">> new_length = 0


def new_length (x):
    if x.StartEnd=="S":
        A = x.END_MP - x.BEGINNINGMILE
        return A
    elif x.StartEnd=="E":
        B = x.ENDINGMILE - x.BEGIN_MP
        return B
    elif x.StartEnd=="M":
        C = x.ENDINGMILE - x.BEGINNINGMILE
        return C
    else:
        return 0

In [ ]:
#Cell 7
#Adding column for new_length


Roadway_EEC['NewLength']=Roadway_EEC.apply (new_length, axis = 1)
Roadway_EEC.head ()

In [ ]:
# Cell 8

#Adding a new column for weighted EEC
# Calculating EEC for the new_length. Weighted EEC = (new_length/length of the segment)*EEC of the segment


Roadway_EEC ["Weighted_EEC"] = (Roadway_EEC["NewLength"]/Roadway_EEC["LENGTH"]) * Roadway_EEC ["Road_EEC"]
Roadway_EEC.head ()

In [ ]:
# Cell 9
# A new list of EEC with existing EEC and Weighted EEC

def new_EEC (x):
    if x.Weighted_EEC ==0:
        C = x.Road_EEC
        return C
    else:
        D = x.Weighted_EEC
        return D

In [ ]:
# Cell 10
#New Column for Existing and Weighted EEC

Roadway_EEC ["NEW_EEC"] = Roadway_EEC.apply (new_EEC, axis = 1)
Roadway_EEC.head ()

In [ ]:
#Cell 11
#Taking the summation of segment EECs for each project
#Project_No is unique for every project

Roadway_EEC_new = Roadway_EEC['NEW_EEC'].groupby(Roadway_EEC['Project_No']).sum ().to_frame(name='Roadway_EEC')
Roadway_EEC_new.head ()

In [ ]:
#Cell 12
Roadway_EEC_new['Roadway_EEC'] = Roadway_EEC_new['Roadway_EEC'].replace(np.nan, 0)
Roadway_EEC_new.head ()

## Intersection EEC

In [ ]:
#Cell 13
#Importing the spatially joined Intersection database

Intersection_EEC = pd.read_csv ("Intersection_EEC.csv", encoding ="latin-1")
Intersection_EEC.head ()

In [ ]:
#Cell 14
#Taking the summation of Intersection EECs for each project
Intersection_EEC_new = Intersection_EEC['Int_EEC'].groupby(Intersection_EEC['Project_No']).sum ().to_frame(name='Intersection_EEC')
Intersection_EEC_new['Intersection_EEC'] = Intersection_EEC_new['Intersection_EEC'].replace(np.nan, 0)
Intersection_EEC_new.head ()

In [ ]:
# #Cell 15
# #Merging the Intersection EEC database with the Roadway EEC database

# Road_Int_EEC = pd.merge (Intersection_EEC_new,Roadway_EEC_new,left_on = "Project_No",right_on = "Project_No", how = "outer")
# Road_Int_EEC.head ()

## Ramp EEC

In [ ]:
#Cell 15
#Importing the spatially joined Ramp database

Ramp_EEC = pd.read_csv ("Ramp_EEC.csv", encoding ="latin-1")
Ramp_EEC.head ()

In [ ]:
#Cell 16
#Taking the summation of Ramp EECs for each project
Ramp_EEC_new = Ramp_EEC['Ramp_EEC'].groupby(Ramp_EEC['Project_No']).sum ().to_frame(name='Ramp_EEC')
Ramp_EEC_new['Ramp_EEC'] = Ramp_EEC_new['Ramp_EEC'].replace(np.nan, 0)
Ramp_EEC_new.head ()

## Merging Projects, Roadway_EEC, Intersection_EEC and Ramp_EEC database

In [ ]:
#Cell 17
#Importing the file with project details.

Projects = pd.read_csv ("Projects.csv", encoding ="latin-1")
Projects.head ()

In [ ]:
#Cell 18
#Merge 1: Projects and Roadway_EEC


Merge1 = pd.merge (Projects,Roadway_EEC_new,left_on = "Project_No",right_on = "Project_No", how = "outer")
Merge1.head ()

In [ ]:
#Cell 19
#Merge 2: Merge1 and Intersection EEC


Merge2 = pd.merge (Merge1,Intersection_EEC_new,left_on = "Project_No",right_on = "Project_No", how = "outer")
Merge2.head ()

In [ ]:
#Cell 20
#Merge 2: Merge1 and Intersection EEC


Merge3 = pd.merge (Merge2,Ramp_EEC_new,left_on = "Project_No",right_on = "Project_No", how = "outer")
Merge3 = Merge3.replace(np.nan, 0)
Merge3.head ()

In [ ]:
#Cell 21
#Total EEC of a project =  Roadway EEC + Intersection EEC + Ramp_EEC


Merge3 ["Final_EEC"] = Merge3 ["Intersection_EEC"] + Merge3 ["Roadway_EEC"] + Merge3 ["Ramp_EEC"]
Merge3.head ()

## Sorting and Ranking

In [ ]:
#Cell 22
#Sorting the EECs from highest to lowest

FINAL_EEC_sorted = Merge3.sort_values(by= "Final_EEC", ascending=False)
FINAL_EEC_sorted.head ()

In [ ]:
#Cell 23
#Adding Rank

FINAL_EEC_sorted["Rank"] = FINAL_EEC_sorted["Final_EEC"].rank(ascending=False)
FINAL_EEC_sorted.head ()

In [ ]:
# Cell 24
FINAL_EEC_sorted.to_csv ("Final EEC List.csv")

## Additional Analysis
### Summary Statistics

In [ ]:
#Cell 25
FINAL_EEC_sorted["Final_EEC"].describe ()

### Sorting by sign of EEC

In [ ]:
#Cell 26
def SIGN (x):
    if x.Final_EEC >0:
        return "P"
    elif x.Final_EEC <0:
        return "N"
    else:
        return "0"

In [ ]:
#Cell 27
FINAL_EEC_sorted ["Sign"] = FINAL_EEC_sorted.apply (SIGN, axis = 1)
FINAL_EEC_sorted.head ()

In [ ]:
#Cell 28
FINAL_EEC_sorted["Sign"].value_counts ()
